In [90]:
# Import Libraries
import requests
import json
import time
from datetime import datetime
import csv

# Relevant Subreddits for collection
# r/Ethereum
# r/EthTrader
# r/EthInvestor

In [81]:
# Using Chase's Reddit Account
hdr = {'User-Agent': 'windows:r/ethereum.single.result:v1.0' +
       '(by /u/Budabellly)'}
url = 'https://www.reddit.com/r/ethereum/.json'
req = requests.get(url, headers=hdr)
json_data = json.loads(req.text)


In [82]:
# # Example of request
# posts = json.dumps(json_data['data']['children'], indent=4, sort_keys=True)
# print(posts)

In [83]:
data_all = json_data['data']['children']
num_of_posts = 0
time_start = time.time()
while len(data_all) <= 750:
    time.sleep(2)
    last = data_all[-1]['data']['name']
    url = 'https://www.reddit.com/r/ethereum/.json?after=' + str(last)
    req = requests.get(url, headers=hdr)
    data = json.loads(req.text)
    data_all += data['data']['children']
    if num_of_posts == len(data_all):
        print num_of_posts, len(data_all)
        break
    else:
        num_of_posts = len(data_all)
print "Time: ", time.time() - time_start

571 571
Time:  54.1468839645


In [67]:
# print json.dumps(data_all, indent=4, sort_keys=True)
print len(data_all), type(data_all)

568 <type 'list'>


In [108]:
# Helper functions 

def find_earliest(data):
    times = [item["created_utc"] for item in data]
    earliest = min(times)
    print "\nTime of earliest post: ", datetime.fromtimestamp(earliest).strftime('%c')

def type_fix(value):
    if type(value) == type(1) or type(value) == type(1.):
        return float(value)
    else:
        return unicode(value).encode('utf-8')

In [110]:
wanted_keys = ["domain", "title", "media_embed", "likes", "view_count", "name", "author", "score", "selftext", "created", "created_utc", "permalink", "num_comments"]
print "JSON: ", type(data_all[0]), len(data_all[0].keys()), data_all[0].keys()
item = data_all[567]["data"]
print "Number of Features before clean: ", len(item.keys())
# print json.dumps(data_all[0], indent = 2)

data = [d["data"] for d in data_all]
new_data = []
for i, item in enumerate(data):
    new_item = {k:type_fix(v) for k,v in item.iteritems() if k in wanted_keys}
    new_data.append(new_item)
    
print "Number of Features after clean: ", len(new_data[0].keys()),'\n\nCleaned Features: ',new_data[0].keys()
find_earliest(new_data)

with open("reddit-ethereum.csv", "a") as f:
    w = csv.DictWriter(f, new_data[0].keys())
    w.writeheader()
    w.writerows(new_data)
#     for item in new_data:
#         myfile.write('\n' + )
        

# new_item = {k:v for k,v in item.iteritems() if k in wanted_keys}
# print "Number of Features after clean: ", len(new_item.keys())
# print '\n', "Cleaned Features: ", new_item.keys()

JSON:  <type 'dict'> 2 [u'kind', u'data']
Number of Features before clean:  70
Number of Features after clean:  13 

Cleaned Features:  [u'view_count', u'domain', u'score', u'title', u'num_comments', u'created', u'created_utc', u'media_embed', u'permalink', u'selftext', u'likes', u'author', u'name']

Time of earliest post:  Thu Oct 19 16:12:57 2017
